In [22]:
# Imports 
import os
import pyfits
import time
import datetime
import scipy.misc
import shutil
import numpy as np
import posixpath
import matplotlib.pyplot as plt
import imageio
from PIL import Image
from astropy.io import fits
%matplotlib inline

from breadboard import BreadboardClient
bc = BreadboardClient(config_path='D:\\User\Documents\\My Programs\\API_CONFIG_fermi3.json')


import warnings
warnings.filterwarnings("ignore", "Your application has authenticated using end user credentials")
warnings.filterwarnings("ignore", "Could not find appropriate MS Visual C Runtime")


# Helper Functions

In [23]:
def select_today(main_folder="\\\\bec1server\\Raw Data\\Fermi3\\Images"):
    '''
    Get today's folder
    ===============================
    '''
    start_time = datetime.datetime.now() 
    start_year = start_time.strftime('%Y')
    start_month = start_time.strftime('%Y-%m')
    start_date = start_time.strftime('%Y-%m-%d')
    output_folder = os.path.join(main_folder, start_year, start_month, start_date)
    if not (os.path.exists(output_folder)): os.makedirs(output_folder)
    return output_folder


def plot_image(imgs, clim=[0, 1.5]):
    '''
    Plot an image
    =============================
    ''' 
    If = imgs[0] - imgs[2]
    Ii = imgs[1] - imgs[2] 
    od = - np.log(If / Ii) 
    plt.figure(figsize=[10, 10])
    plt.imshow(od, origin=0, clim=clim)
    plt.title("{}".format(output_filename))
    plt.tight_layout()
    plt.show()
    return Null

def write_jpeg(imgs, clim=[0, 2]):
    '''
    Write a jpeg of the OD
    =============================
    ''' 
    od_folder = select_today(main_folder="\\\\bec1server\\web\\odimages\\Fermi3")
    od_filepath = os.path.join(od_folder, os.path.splitext(output_filename)[0]+'.jpg')
    If = imgs[0] - imgs[2]
    Ii = imgs[1] - imgs[2] 
    with np.errstate(divide='ignore', invalid='ignore'): od = - np.log(If / Ii) 
    newim = Image.fromarray(od*255/2)
    newim = newim.convert("L")
    newim.save(od_filepath)
    return od_filepath


def upload_odimage(source_file_name):
    """"""
    '''
    Uploads a file to the google cloud odimages bucket. This is for a future website
    =============================
    ''' 

    start_time = datetime.datetime.now() 
    start_year = start_time.strftime('%Y')
    start_month = start_time.strftime('%Y-%m')
    destination_blob_name = "Fermi3/" + start_year + "/" + start_month + "/" + os.path.splitext(output_filename)[0] + '.jpg'
    
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    blob.make_public()

    return blob.public_url


def getFileList(folder = 'Not Provided'):
    '''
    Get a list of files in a folder
    ===============================
    '''
    # Confirm that given folder path is correct
    if not os.path.exists(folder): raise ValueError("Folder '{}' doesn't exist".format(folder))
    # Folder contents
    filenames = [filename for filename in os.listdir(folder)]
    # Output
    paths = [os.path.join(folder,f) for f in filenames]
    return (filenames, paths)

# Main Code

In [24]:
# Global settings 
refresh_time = 0.3 # seconds
main_folder = "\\\\bec1server\\Raw Data\\Fermi3\\Images"
input_format = '%Y-%m-%d_%H-%M-%S'
start_time = datetime.datetime.now()
today_folder = select_today()
timecounter = 8 * refresh_time
storage_client = storage.Client() # for uploading odimages to google cloud (Future website)
bucket = storage_client.bucket("odimages") # for uploading odimages to google cloud (Future website)
names_old, paths_old = getFileList(today_folder)

In [ ]:

# Main Loop 
while True: 
    today_folder = select_today()
    # Get a list of all the images in the folder 
    names, paths = getFileList(today_folder)
    names = sorted(names)
    paths = sorted(paths)
    
    # check if a new image has come in
    if len(names)>len(names_old):
        timecounter = 0
        output_filename = names[-1]
        output_filepath = paths[-1]
        print(output_filename)
        # Write a JPEG of the OD
        try:
            with fits.open(output_filepath) as hdul:
                local_od_filepath = write_jpeg(hdul[0].data)
                cloud_od_filepath = upload_odimage(local_od_filepath)
        except: pass
        # Write to Breadboard
        try:
            resp = bc.post_images(
                        image_names = os.path.splitext(output_filename)[0],
                        filepath = output_filepath,
                        odpath = cloud_od_filepath,
                        )
        except: pass
        names_old = names
        
    # Ensure runs are matched with a second post request:
    if timecounter == 7 * refresh_time:
        try:
            resp = bc.post_images(image_names = os.path.splitext(output_filename)[0],
                            filepath = output_filepath,
                            odpath = cloud_od_filepath,
                            force_match=True) 
            if resp.status_code!=200:
                print(resp.text)
        except: pass
        
    # Wait before checking again 
    time.sleep(refresh_time)
    timecounter += refresh_time

2020-02-21_13-14-17.fits
2020-02-21_13-14-56.fits
2020-02-21_13-15-36.fits


# Tests